# EXPLORATION 3. 뉴스기사 크롤링 및 분류

---

### 데이터 정제

In [1]:
import os
import pandas as pd

csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


In [2]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
5119    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
5120    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
5121    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
5122    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
5123    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 5124, dtype: object

In [3]:
# null 값이 있는지 확인
print(df.isnull().sum())

news    0
code    0
dtype: int64


In [4]:
# 중복된 샘플들을 제거합니다.
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  3994


### 데이터 탐색

In [5]:
print(df.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학    903
1     사회   1668
2  생활/문화   1423


### 토큰화

In [133]:
from konlpy.tag import Mecab
mecab = Mecab()

### 불용어(stopwords) 제거

In [8]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [135]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = mecab.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords] 
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))

    return text_data

In [136]:
text_data = preprocessing(df['news'])
print(text_data[0])

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 인 이상 가구 기준 으로 만 원 받 게 되 며 인 가구 만 원 인 가구 만 원 인 가구 만 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 시 이미 모든 시민 경기도 파주시 재난 기본 소득 인 당 각 만 원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만 원 경기도 재난 기본소득 만 원 정부 긴급 재난 지원금 만 원 총 만 원 받 게 된다 취약 계층 아닌 시민 오 월일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 를 구성 해 긴급 재난 지원금 원활 게 지급 될 도록 지원 한다 권 자 재 배포


### Naive bayes

In [137]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [138]:
# 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

In [139]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 2995
테스트용 뉴스 기사의 개수 :  999
훈련용 레이블의 개수 :  2995
테스트용 레이블의 개수 :  999


In [140]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.def tfidf_vectorizer(data):
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [141]:
def tfidf_vectorizer(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

In [161]:
new_sent = preprocessing(["민주당 일각에서 법사위의 체계·자구 심사 기능을 없애야 한다는 \
                           주장이 나오는 데 대해 “체계·자구 심사가 법안 지연의 수단으로 \
                          쓰이는 것은 바람직하지 않다”면서도 “국회를 통과하는 법안 중 위헌\
                          법률이 1년에 10건 넘게 나온다. 그런데 체계·자구 심사까지 없애면 매우 위험하다”고 반박했다."])
print(clf.predict(tfidf_vectorizer(new_sent)))

['사회']


In [162]:
new_sent = preprocessing(["인도 로맨틱 코미디 영화 <까립까립 싱글>(2017)을 봤을 때 나는 두 눈을 의심했다. \
                          저 사람이 남자 주인공이라고? 노안에 가까운 이목구비와 기름때로 뭉친 파마머리와, \
                          대충 툭툭 던지는 말투 등 전혀 로맨틱하지 않은 외모였다. 반감이 일면서 \
                          ‘난 외모지상주의자가 아니다’라고 자부했던 나에 대해 회의가 들었다.\
                           티브이를 꺼버릴까? 다른 걸 볼까? 그런데, 이상하다. 왜 이렇게 매력 있지? 개구리와\
                            같이 툭 불거진 눈망울 안에는 어떤 인도 배우에게서도 느끼지 못한 \
                            부드러움과 선량함, 무엇보다 슬픔이 있었다. 2시간 뒤 영화가 끝나고 나는 완전히 이 배우에게 빠졌다"])
print(clf.predict(tfidf_vectorizer(new_sent)))

['생활/문화']


In [163]:
new_sent = preprocessing(["20분기 연속으로 적자에 시달리는 LG전자가 브랜드 이름부터 성능, 디자인까지 대대적인 변화를 \
                          적용한 LG 벨벳은 등장 전부터 온라인 커뮤니티를 뜨겁게 달궜다. 사용자들은 “디자인이 예쁘다”, \
                          “슬림하다”는 반응을 보이며 LG 벨벳에 대한 기대감을 드러냈다."])
print(clf.predict(tfidf_vectorizer(new_sent)))

['IT/과학']


In [142]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.81      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.80      0.81       999
weighted avg       0.82      0.82      0.82       999



## Step 1. 형태소 분석기 변경해보기

### Hannanum

In [165]:
from konlpy.tag import Hannanum
hannanum = Hannanum()

# 토큰화 및 불용어 제거
def preprocessing_hannanum(data):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = hannanum.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords] 
        text_data.append(temp_data)

        text_data = list(map(' '.join, text_data))

    return text_data

In [ ]:
text_data_hannanum = preprocessing_hannanum(df['news'])
print(text_data_hannanum[0])

### Naive bayes

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [30]:
# 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data_hannanum, df['code'], random_state = 0)

In [31]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect_hannanum = CountVectorizer()
X_train_counts_hannanum = count_vect_hannanum.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer_hannanum = TfidfTransformer()
X_train_tfidf_hannanum = tfidf_transformer_hannanum.fit_transform(X_train_counts_hannanum)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf_hannanum = MultinomialNB().fit(X_train_tfidf_hannanum, y_train)

In [32]:
def tfidf_vectorizer_hannanum(data):
    data_counts = count_vect_hannanum.transform(data)
    data_tfidf = tfidf_transformer_hannanum.transform(data_counts)
    return data_tfidf

In [37]:
y_pred = clf_hannanum.predict(tfidf_vectorizer_hannanum(X_test))
print("Hannanum")
print()
print(metrics.classification_report(y_test, y_pred))

Hannanum

              precision    recall  f1-score   support

       IT/과학       0.90      0.64      0.75       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.79      0.79      0.79       341

    accuracy                           0.81       999
   macro avg       0.83      0.78      0.80       999
weighted avg       0.82      0.81      0.81       999



### Kkma

In [6]:
from konlpy.tag import Kkma
kkma = Kkma()

# 토큰화 및 불용어 제거
def preprocessing_kkma(data):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = kkma.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords] 
        text_data.append(temp_data)

        text_data = list(map(' '.join, text_data))

    return text_data

In [ ]:
text_data_kkma = preprocessing_kkma(df['news'])
print(text_data_kkma[0])

### Naive bayes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
# 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data_kkma, df['code'], random_state = 0)

In [ ]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect_kkma = CountVectorizer()
X_train_counts_kkma = count_vect_kkma.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer_kkma = TfidfTransformer()
X_train_tfidf_kkma = tfidf_transformer_kkma.fit_transform(X_train_counts_kkma)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf_kkma = MultinomialNB().fit(X_train_tfidf_kkma, y_train)

In [ ]:
def tfidf_vectorizer_kkma(data):
    data_counts = count_vect_kkma.transform(data)
    data_tfidf = tfidf_transformer_kkma.transform(data_counts)
    return data_tfidf

In [ ]:
y_pred = clf_kkma.predict(tfidf_vectorizer_kkma(X_test))
print("Kkma")
print()
print(metrics.classification_report(y_test, y_pred))

### Komoran

In [8]:
from konlpy.tag import Komoran
komoran = Komoran()

# 토큰화 및 불용어 제거
def preprocessing_komoran(data):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = komoran.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords] 
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))

    return text_data

In [9]:
text_data_komoran = preprocessing_komoran(df['news'])
print(text_data_komoran[0])

파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 파이낸셜뉴스 강 근 주 파주시 일 관내 취약 계층 만 가구 대하 아 정부 긴급 재난 지원금 입금 완료 았 파주시 민이 받 긴급 재난 지원금 인 이상 가구 기준 으로 만원 받 게 되 며 인 가구 만원 인 가구 만원 인 가구 만원 이다 정부 발표 ㄴ 긴급 재난 지원금 파주시 민 지급 ㄴ 금액 다른 이유 국비 지방비 부담 비율 이다 파주시 이미 모든 시민 경기도 파주시 재난 기본소득 인당 각 만원 지급 고 어 시민 국비 지원금 만 지급 며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만원 경기도 재난 기본소득 만원 정부 긴급 재난 지원금 만원 총 만원 받 게 되 ㄴ 취약 계층 아니 ㄴ 시민 오 월 일 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 ㄹ 세대주 가족 지원금 일괄 신청 아야 한편 파주시 일 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태스크포스를 구성 아 긴급 재난 지원금 원활 게 지급 되 ㄹ 도록 지원 ㄴ다 저작권 자 파이낸셜뉴스 재 배포


### Naive bayes

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [11]:
# 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data_komoran, df['code'], random_state = 0)

In [12]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect_komoran = CountVectorizer()
X_train_counts_komoran = count_vect_komoran.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer_komoran = TfidfTransformer()
X_train_tfidf_komoran = tfidf_transformer_komoran.fit_transform(X_train_counts_komoran)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf_komoran = MultinomialNB().fit(X_train_tfidf_komoran, y_train)

In [13]:
def tfidf_vectorizer_komoran(data):
    data_counts = count_vect_komoran.transform(data)
    data_tfidf = tfidf_transformer_komoran.transform(data_counts)
    return data_tfidf

In [14]:
y_pred = clf_komoran.predict(tfidf_vectorizer_komoran(X_test))
print("Komoran")
print()
print(metrics.classification_report(y_test, y_pred))

Komoran

              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.80      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.81      0.82       999
weighted avg       0.83      0.82      0.82       999



### Okt

In [15]:
from konlpy.tag import Okt
okt = Okt()

# 토큰화 및 불용어 제거
def preprocessing_okt(data):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = okt.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords] 
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))

    return text_data

In [16]:
text_data_okt = preprocessing_okt(df['news'])
print(text_data_okt[0])

파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 파이낸셜뉴스 강 근 주 파주시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지 원금 입금 완료 했다 파주시민 받는 긴급 재난 지원 금은 인 이상 가구 기준 으로 만원 받게 되며 인 가구 만원 인 가구 만원 인 가구 만원 이다 정부 발표 긴급 재난 지 원금 파주시민 지급 금액 다른 이유 국비 지 방비 부담 비율 이다 파주시 이미 모든 시민 경기도 파주시 재난 기본소득 인 당 각 만원 지급 하고 있어 시민 국비 지 원금 만 지급 하며 인 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 만원 경기도 재난 기본소득 만원 정부 긴급 재난 지 원금 만원 총 만원 받게 된다 취약 계층 아닌 시민 오는 월일 부터 소지 하고 있는 신용 체크카드 사 홈페이지 에서 긴급 재난 지 원금 지원 신청 있다 세대주 가족 지 원금 일괄 신청 해야 한다 한편 파주시 일 김정기 부시장 단장 으로 하는 긴급 재난 지 원금 추진 태스크포스 를 구성 해 긴급 재난 지 원금 원활하게 지급 될 있도록 지원 저작권 자 파이낸셜뉴스 전 재재 배포


### Naive bayes

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [18]:
# 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data_okt, df['code'], random_state = 0)

In [19]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect_okt = CountVectorizer()
X_train_counts_okt = count_vect_okt.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer_okt = TfidfTransformer()
X_train_tfidf_okt = tfidf_transformer_okt.fit_transform(X_train_counts_okt)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf_okt = MultinomialNB().fit(X_train_tfidf_okt, y_train)

In [20]:
def tfidf_vectorizer_okt(data):
    data_counts = count_vect_okt.transform(data)
    data_tfidf = tfidf_transformer_okt.transform(data_counts)
    return data_tfidf

In [21]:
y_pred = clf_okt.predict(tfidf_vectorizer_okt(X_test))
print("Okt")
print()
print(metrics.classification_report(y_test, y_pred))

Okt

              precision    recall  f1-score   support

       IT/과학       0.91      0.70      0.79       236
          사회       0.78      0.93      0.85       422
       생활/문화       0.83      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.80      0.81       999
weighted avg       0.83      0.82      0.82       999



## Step 2. 불용어 추가해보기

### 불용어 추가

In [100]:
new_stopwords = stopwords[:]
print(new_stopwords)

['에', '는', '은', '을', '했', '에게', '있', '이', '의', '하', '한', '다', '과', '때문', '할', '수', '무단', '따른', '및', '금지', '전재', '경향신문', '기자', '는데', '가', '등', '들', '파이낸셜', '저작', '등', '뉴스']


In [101]:
file_path1 = os.getenv("HOME") + "/aiffel/news_crawler/EOMI2.TXT"

with open(file_path1) as file_object:
    contents = file_object.read()

for c in contents:
    if c != '\n' and c != '\x1a':
        new_stopwords.append(c)

print(len(new_stopwords))

2299


In [102]:
file_path2 = os.getenv("HOME") + "/aiffel/news_crawler/JOSA2.TXT"

with open(file_path2) as file_object:
    contents = file_object.read()

for c in contents:
    if c != '\n' and c != '\x1a':
        new_stopwords.append(c)

print(len(new_stopwords))


3693


### 토큰화

In [103]:
# from konlpy.tag import Okt
# okt = Okt()

In [104]:
from konlpy.tag import Mecab
mecab = Mecab()

In [105]:
# 토큰화 및 불용어 제거
def preprocessing_new(data):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = mecab.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in new_stopwords] 
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))

    return text_data

In [106]:
text_data_new = preprocessing_new(df['news'])
print(text_data_new[0])

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 관내 취약 계층 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 이상 가구 기준 으로 원 받 가구 원 가구 원 가구 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 이미 모든 시민 경기도 파주시 재난 기본 소득 당 각 원 지급 시민 국비 지원금 지급 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 원 경기도 재난 기본소득 원 정부 긴급 재난 지원금 원 총 원 받 된다 취약 계층 아닌 시민 월일 부터 소지 신용 체크카드 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 구성 해 긴급 재난 지원금 원활 지급 될 도록 지원 한다 권 재 배포


### Naive bayes

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [108]:
# 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data_new, df['code'], random_state = 0)

In [109]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect_new = CountVectorizer()
X_train_counts_new = count_vect_new.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer_new = TfidfTransformer()
X_train_tfidf_new = tfidf_transformer_new.fit_transform(X_train_counts_new)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf_new = MultinomialNB().fit(X_train_tfidf_new, y_train)

In [110]:
def tfidf_vectorizer_new(data):
    data_counts = count_vect_new.transform(data)
    data_tfidf = tfidf_transformer_new.transform(data_counts)
    return data_tfidf

In [111]:
y_pred = clf_new.predict(tfidf_vectorizer_new(X_test))
print("(new stopwords)")
print()
print(metrics.classification_report(y_test, y_pred))

(new stopwords)

              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.81      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.80      0.81       999
weighted avg       0.82      0.82      0.82       999



## Step 3. 다른 날짜 데이터 추가해보기

### 데이터 정제

In [112]:
csv_path1 = os.getenv("HOME") + "/aiffel/news_crawler/news_data.csv"
csv_path2 = os.getenv("HOME") + "/aiffel/news_crawler/news_data2.csv"

df1 = pd.read_table(csv_path1, sep=',')
df2 = pd.read_table(csv_path2, sep=',')

df_concat = pd.concat([df1,df2])

In [113]:
df_concat

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회
...,...,...
3698,레고켐바이오사이언스(대표 김용주·사진)가 글로벌 기술수출에 또다시 성공했다. 201...,IT/과학
3699,제21대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 15일 오...,IT/과학
3700,아마존 [AFP=연합뉴스 자료사진] 아마존 [AFP=연합뉴스 자료사진]\n\n직원들...,IT/과학
3701,"제21대 국회의원선거가 임박한 가운데, 투표 및 개표 방송을 준비하는 기업들의 움직...",IT/과학


In [114]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df_concat['news'] = df_concat['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df_concat['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
3698    레고켐바이오사이언스대표 김용주사진가 글로벌 기술수출에 또다시 성공했다 년 중국 포순...
3699    제대 국회의원선거 기간 중단됐던 네이버 실시간 급상승 검색어 서비스가 일 오후 시부...
3700    아마존 연합뉴스 자료사진 아마존 연합뉴스 자료사진직원들 코로나 확산 방지 조치 충분...
3701    제대 국회의원선거가 임박한 가운데 투표 및 개표 방송을 준비하는 기업들의 움직임이 ...
3702    롤 점검이 일 진행될 예정이다온라인게임 리그오브레전드이하 롤은 일 오전 시부터 오전...
Name: news, Length: 8827, dtype: object

In [115]:
# 중복된 샘플들을 제거합니다.
df_concat.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df_concat))

뉴스 기사의 개수:  6127


### 데이터 탐색

In [116]:
print(df_concat.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학   1138
1     경제    901
2     사회   2222
3  생활/문화   1866


### 토큰화

In [117]:
# from konlpy.tag import Okt
# okt = Okt()

In [118]:
from konlpy.tag import Mecab
mecab = Mecab()

In [119]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing_concat(data):
    text_data = []

    for sentence in data:
        temp_data = []
        #- 토큰화
        temp_data = mecab.morphs(sentence) 
        #- 불용어 제거
        temp_data = [word for word in temp_data if not word in new_stopwords] 
        text_data.append(temp_data)

    text_data = list(map(' '.join, text_data))

    return text_data

In [120]:
text_data_concat = preprocessing_concat(df_concat['news'])
print(text_data_concat[0])

파주 시청 사진제 공파 주시 파주 시청 사진 제공 파주시 파주 강근주 파 주 관내 취약 계층 가구 대해 정부 긴급 재난 지원금 입금 완료 다파 주 시민 받 긴급 재난 지원금 이상 가구 기준 으로 원 받 가구 원 가구 원 가구 원 정부 발표 긴급 재난 지원금 파주 시민 지급 금액 다른 이유 국비 지방비 부담 비율 다파 주 이미 모든 시민 경기도 파주시 재난 기본 소득 당 각 원 지급 시민 국비 지원금 지급 가구 기준 으로 총 지원 금액 파주시 재난 기본소득 원 경기도 재난 기본소득 원 정부 긴급 재난 지원금 원 총 원 받 된다 취약 계층 아닌 시민 월일 부터 소지 신용 체크카드 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 해야 한다 한편 파 주 김정기 부시장 단장 으로 긴급 재난 지원금 추진 태 스 크 포스 구성 해 긴급 재난 지원금 원활 지급 될 도록 지원 한다 권 재 배포


In [121]:
print(text_data_concat[6])

코로나 걸렸 친구 아무래도 좀 멀리 될 아요 재발 가능 성 그리고 누가 기침 해도 쟤 코로나다 라고 놀리 거나 코로나 왕따 당하 학생 생길 어요 올해 고등학교 진학 모양 등교 시작 아무래도 전 보단 예민 상태 학교 다닐 다며 같이 이양 아니 등교 개학 시작 약 주일 앞두 학생 사이 에선 코로나 왕따 우려 번지 이날 사회관 계망 서비스 에선 등교 개학 문제점 시물 공유 여기 이동 수업 이나 급식 시간 중 감염 우려 아니 따돌림 항목 포함 돼 선풍기 이용 화장실 무조건 참 아야 함 일부 내용 정확 정보 바탕 으로 작성 된 지만 코로나 왕따 코로나 환자 대한 사회 적 낙인 배제 학교 에서 발생 다는 현실 적 우려 구체 적 으로 천식 감기 학생 기침 시선 쏠릴 개학 명 코 걸려서 전염 왕따 가능 성 담겼 특히 천식 호흡기 기저 질환 앓 학생 누구 보다 걱정 크 다고 전 모양 오랜만 등교 앞두 조금 설렌다 온라인 수업 보다 직접 수업 듣 이해 된다면서 천식 기침 조절 없 기침 친구 시선 좋 않 다고 우려 역시 천식 중 김모양 호흡 곤란 자주 와서 장시간 마스크 쓰 답답 어지러운 잔기침 때 쏠리 시선 불편 다고 사실상 학기 시작 이나 마찬가지 등교 개학 이후 어떻게 친구 사귀 어야 할지 고민거리 모양 코로나 언제 어디 걸리 는지 모르 학교 모여 의심 환자 될까 우려 된다 선생 님 말씀 안 듣 코 옮긴다고 장난치 애 을까 걱정 라고 김서영 박채영 재 배포


### Naive bayes

In [122]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [123]:
# 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data_concat, df_concat['code'], stratify=df_concat['code'], test_size=0.2,random_state = 0)

In [124]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 4901
테스트용 뉴스 기사의 개수 :  1226
훈련용 레이블의 개수 :  4901
테스트용 레이블의 개수 :  1226


In [125]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect_concat = CountVectorizer()
X_train_counts_concat = count_vect_concat.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer_concat = TfidfTransformer()
X_train_tfidf_concat = tfidf_transformer_concat.fit_transform(X_train_counts_concat)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.def tfidf_vectorizer(data):
clf_concat = MultinomialNB().fit(X_train_tfidf_concat, y_train)

In [126]:
def tfidf_vectorizer_concat(data):
    data_counts = count_vect_concat.transform(data)
    data_tfidf = tfidf_transformer_concat.transform(data_counts)
    return data_tfidf

In [127]:
y_pred = clf_concat.predict(tfidf_vectorizer_concat(X_test))
print("add data")
print(metrics.classification_report(y_test, y_pred))

add data
              precision    recall  f1-score   support

       IT/과학       0.84      0.69      0.76       228
          경제       0.91      0.49      0.64       180
          사회       0.73      0.90      0.81       445
       생활/문화       0.73      0.76      0.74       373

    accuracy                           0.76      1226
   macro avg       0.80      0.71      0.74      1226
weighted avg       0.77      0.76      0.75      1226

